<a href="https://colab.research.google.com/github/brunatoloti/data-science-alura-cursos/blob/main/Machine%20Learning%3A%20Valida%C3%A7%C3%A3o%20de%20Modelos/machine_learning_validacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning: Validação de Modelos

Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

Importando a base de dados

In [ ]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


## A influência da aleatoriedade na validação do modelo

Definindo x e y e separando os dados entre treino e teste. Além disso, definindo o SEED

In [ ]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


Utilizando um classificador Dummy para decidir uma taxa de acerto aceitável

In [ ]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


Agora, rodando um classificador de Árvore de Decisão, com profundidade máxima igual a 2

In [ ]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


Vemos que a Árvore de Decisão, nessas condições, obteve uma acurácia maior do que o classificador Dummy, o que é muito bom!

Mas, e se mudarmos o SEED?

Fazendo esse teste com um novo SEED

In [ ]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


A acurácia, mudando o SEED de 158020 para 5, aumentou!

Então, aquela decisão de julgar isso como bom ou ruim em comparação ao que obtínhamos com o classificador Dummy está melhor, entretanto, ainda estamos vulneráveis a tomar uma decisão baseada em uma aleatoriedade. Não faz sentido validar, julgar se é bom ou ruim, de acordo com um número aleatório. Queremos minimizar o efeito dessa aleatoriedade, dessa escolha aleatória do treino, do teste e da classificação de Árvore de Decisão, na nossa decisão.

No entanto, se rodamos o treino e o teste uma única vez, corremos o risco de os ter separado mal. Precisamos de outra forma de treinar e testar mais de uma vez para que não tenhamos uma estimativa única, uma maneira que permita rodar o treino e o teste diversas vezes e, a partir disso, obter uma estimativa.

Em vez de um ponto, queremos um intervalo. Precisamos encontrar um método, um algoritimo para isso. 

## Usando e avaliando com Cross Validation

In [ ]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Acurácia com cross validation 3 =  [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Acurácia com cross validation 3 =  [74.99, 76.57]


Se mudarmos o valor de SEED — número aleatório que influencia o algoritmo —, a acurácia se mantém. Podemos alterar o SEED para qualquer valor e o intervalo permanecerá o mesmo, de 74.99 a 76.57. Ou seja, nosso algoritmo não está mais tão suscetível a essa aleatoriedade! (Podemos testar com SEEDs diferentes para ver que, de fato, isso ocorre!). Contudo, estamos suscetíveis ao número de cv.

Dependendo do número de cv (que é a quantidade de vezes que será feito o treino e teste), podemos ter um intervalo diferente. Por exemplo, fazendo o mesmo código para cv=10.

In [ ]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Acurácia com cross validation 10 =  [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Acurácia com cross validation 10 =  [74.24, 77.32]


Ou, fazendo o mesmo código para cv = 5

In [ ]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Acurácia com cross validation 5 =  [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Acurácia com cross validation 5 =  [75.21, 76.35]


Podemos ver que, de fato, o intervalo depende do valor de cv.

Então, qual número devemos usar para o cross validation (cv)? Alguns papers científicos apontam que escolher 5 ou 10 para cv já é o suficiente. Sendo assim, vamos manter o valor de cv do nosso código como 5, que gerou o intervalo de 75.21 a 76.35, para a taxa de acerto que acreditamos alcançar com o algoritmo que estamos utilizando.

## Aleatoriedade no Cross Validate

O cross_validate() não recebe o parâmetro de aleatoriedade, e esse é o padrão. Da maneira utilizada aqui, ele é determinístico e sabemos em quantos pedaços ele quebrará os dados, porque somos quem decidimos essa quantidade. Portanto, a realidade é que a aleatoriedade (random) só é aplicada em DecisionTreeClassifier(). E, por padrão, ele só a usará em um caso específico e raro. Por estas razões, mesmo que mudemos constantemente o valor de SEED, obteremos basicamente sempre os mesmos números.

A ideia é rodar aleatoriamente, de fato. Por mais que recebamos todos os dados em uma sequência, não queremos quebrar eles em 5 pedaços e, a partir disso, fazer o cross_validate. O ideal seria embaralhar estes dados e, então, executar a validação cruzada. O algoritmo pode fazer isso de maneira mais inteligente, sem embaralhar e copiar todo o espaço de memória e todos os objetos que estiverem dentro, e assim por diante. Mas o importante é a ideia de embaralhar esses dados de alguma maneira, e o cross_validate não faz isso de verdade.

Primeiramente, fazendo uma função para imprimir os resultados, ao invés de sempre ficar colocando os códigos diversas vezes.

In [ ]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print(f"Acurácia média = {round(media*100, 2)}")
    print(f"Acurácia intervalo =  [{round((media - 2 * desvio_padrao)*100, 2)}, {round((media + 2 * desvio_padrao)*100, 2)}]")

O que desejamos modificar é a aleatoriedade, desejamos que ele embaralhe nossos dados, entretanto, não existe um parâmetro para isto. Todavia, o cv aceita como parâmetro, tanto números, quanto geradores de validação cruzada. E existem diversos geradores de validação cruzada. O que estamos usando, por baixo dos panos, é o KFold, que "corta" nossos dados em K pedaços. E o SKLearn possui o KFold.

Para determinarmos o número de vezes de treino e teste, o parâmetro do KFold a ser preenchido é o *n_splits*, ou seja, o cross_validate é um KFold de *n_splits*, nesse caso abaixo igual a 10. Além disso, para embaralhar os dados, usamos o parâmetro *shuffle=True*, pois ele tem um randon_state para ser setado.

In [ ]:
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Acurácia média = 75.78
Acurácia intervalo =  [74.37, 77.19]


Bagunçando/Embaralhando os dados com o parâmetro *shuffle*

In [ ]:
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Acurácia média = 75.76
Acurácia intervalo =  [73.26, 78.26]


Repare que os valores do intervalo são diferentes dos obtidos anteriormente e o valor da média é bem próximo.

Se rodarmos diversas vezes, a acurácia se mantém, pois continua sendo determinística. Mas, ainda assim, é diferente dos valores apresentados sem o *shuffle*. No entanto, repare que a média obtida por meio dos dois algoritmos é bastante próxima, mas o intervalo é diferente. Na prática, se os dados não possuem alguma estrutura interna, como uma sequência por datas, por exemplo, primeiro gera-se um sequência aleatória desses dados e, em seguida, quebra-se em 10 pedaços ou faz-se ambos ao mesmo tempo. Esta é a forma tradicional de execução com *shuffle* e *n_splits*.

## Estratificação com Validação Cruzada

O parâmetro *stratify*  não existe no cross_validate do KFold, que possui apenas os parâmetros *n_splits*, *shuffle* e *random_state*. Sendo assim, não está estratificando.

Então, vamos simular uma situação de azar.

### Simulando situação de azar

Pode ser *azar* como pode ser uma proporção de exemplos desbalanceados entre as classes.

Pegando os dados (dados.sort_values()) do nosso dataframe e ordenando pela classe vendido, crescente (ascending=True), ou seja, primeiro 0 e depois 1. Assim, o "não vendido" virá antes do "vendido". Essa é a separação estabelecida.

In [ ]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


Rodando, primeiramente, sem o parâmetro *shuffle*

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Acurácia média = 57.84
Acurácia intervalo =  [34.29, 81.39]


Note que a acurácia é baixíssima e o intervalo é bem grande!

Rodando, agora, com o parâmetro *shuffle*

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Acurácia média = 75.78
Acurácia intervalo =  [72.3, 79.26]


Já utilizando o *shuffle*, obtemos um resultado suficientemente bom. Ou seja, apenas usando o *shuffle* (embaralhando os dados) em situações desbalanceadas, conseguimos bons resultados.

Porém, e se quiséssemos, mesmo assim, fazer a estratificação?

Podemos usar outro gerador de grupo de separação, o StratifiedKFold, que recebe um número de n_splits, shuffle e depois separa, mantendo a proporção entre as classes.

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Acurácia média = 75.78
Acurácia intervalo =  [73.55, 78.01]


Embora o resultado seja mais fechado, é o mais recomendado. Principalmente quando há um desbalanço entre duas ou mais classes, é interessante utilizar o StratifiedKFold.

## Dados Agrupáveis

Se nós temos diversos carros para os quais o nosso algoritmo, nosso estimador, funcionou bem no passado, quando surgir um novo modelo de carro, o algoritmo funcionará bem para esse último? Não dá pra saber com certeza! Do modo como estamos fazendo, treinamos todos os carros. No treino, nós temos todos os modelos de carro e no teste nós temos todos os modelos de carro aleatorizados e separado por classes, mas não agrupados por modelos. Ou seja, não agrupado pela característica que define o novo tipo de dado. Ou seja, é possível que o algorítimo não generalize bem, para novos carros.

Então, temos que pegar essa característica de agrupamento e tentar trabalhar nos dados. Ou seja, no caso dos carros, trata-se do modelo do veículo. Logo, no momento em que formos treinar, precisaremos agrupar por modelo e separar a validação cruzada por modelos, para depois testar modelos novos.

Contudo, nota-se que nossos dados (preco, vendido, idade_do_modelo e km_por_ano) não têm a categoria modelo_do_carro. Então, vamos ter que gerar uma nova coluna de dados para explorarmos esses dados aleatórios.

### Gerando dados aleatórios

A maneira mais simples seria usar dados.idade_do_modelo, isto é, o modelo do carro seria a idade_do_modelo desse carro. Entretanto, essa é uma premissa falsa, uma vez que nem todos os carros que tem 20 anos necessariamente são do mesmo modelo. Em vez disso, temos que injetar aleatoriedade. Por exemplo, se o carro foi criado há 18 anos, talvez esse carro seja de um modelo A, B ou C. Um modelo do ano 18, 17 ou 19 — com margem um pouco para trás e um pouco para frente. 

In [ ]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=len(dados))
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


Visualizando os valores únicos da coluna *modelo*

In [ ]:
dados.modelo.unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

Note que um dos números é *-1*, o que é meio estranho. Então, fazendo mais algumas modificações na coluna *modelo*.

In [ ]:
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [ ]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

Visualizando quantas vezes cada modelo apareceu na coluna

In [ ]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

### Validação Cruzada usando grupos

Precisamos testar nosso estimador. Para isto, é muito importante pensar qual tipo de teste e treino será utilizado, pois se escolhermos um tipo inapropriado, obteremos uma estimativa irreal de como esse estimador funcionará no mundo real.

O que realmente precisamos é: de capacidade para separar treino e teste na validação cruzada, de acordo com os grupos. Por isso, não podemos utilizar o KFold ou o StratifiedKFold (que foram os que usamos anteriormente), pois eles utilizam todos os modelos de carro para prever todos os modelos de carro (o que é uma situação irreal), não conseguindo lidar da melhor forma quando aparece um novo modelo de carro.

Pesquisando na documentação, podemos ver que o GroupKFold funciona bem para o que queremos.

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média = 75.78
Acurácia intervalo =  [73.67, 77.9]


O valor da média obtida é próximo ao de StratifiedKFold, porque os dados de um modelo de carro estão funcionando de maneria muito parecida, mas isso não é regra. 

Neste caso, de modo geral, os carros se comportam indiferentemente do modelo. Então, nossa classificação foi próxima, levando em consideração ou não o modelo do carro. Quando é dito "levando em consideração" não é para estimar, mas sim para treinar e testar. Lembrando que para classificar só usamos as colunas *preco*, *idade_do_modelo* e *km_por_ano*.

# Pipeline de treino e validação

### Cross Validation com StandardScaler

A árvore de decisão não precisa primeiramente balancear, escalar, as nossas features. Alguns algoritmos precisam, como o SVM. Eles são muito influenciáveis e sensíveis à escala de cada uma das features. 

Se *idade_do_modelo* tem valores entre 0 e 20 e a *preco* tem entre 0 e 90 mil, a influência dessas features em nosso algoritmo, no SVM, por exemplo, será muito diferente entre uma e outra. Então, reescalamos as features quando usamos os algoritmos que são sensíveis à escala, à grandeza dos dados.

Rodando com train_test_split

In [ ]:
scaler = StandardScaler()
scaler.fit(treino_x)
scaler.fit(teste_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

SEED = 301
np.random.seed(SEED)
modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes)*100
print(f"A acurácia foi {round(acuracia, 2)}%")

A acurácia foi 77.48%


Rodando agora com o cross validate

In [ ]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média = 77.27
Acurácia intervalo =  [74.35, 80.2]


Mas, observe acima que não fizemos o scaler dos dados pra rodar com o cross validate. Vamos fazer isso.

Como precisamos de um processo, ou seja, toda vez que for rodar precisa ter escalado a nova porção dos dados de treino (pois não podemos escalar todo o conjunto de dados), usaremos o Pipeline do sklearn.

Além disso, o Pipeline funciona como se fosse um estimador. Logo, podemos passá-lo para o processo de cross validate.

In [ ]:
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x_azar, y_azar, cv=cv, groups=dados.modelo, return_train_score=False)
imprime_resultados(results)

Acurácia média = 76.68
Acurácia intervalo =  [74.28, 79.08]


Para cada um dos processos de fold, rodou tanto a transformacao, quanto o estimador.

Observe que os resultados não ficaram muito distantes uns dos outros, porém, se precisamos reescalar e usar o cross validate, essa é a forma correta.
